[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Olfeng-xalaz/Fallstudie_SCM/blob/Abgabe_SCM_WWI2023A_Gruppe2/Fallstudie.ipynb)

## Imports

In [ ]:
# pip als Paketmanager
! pip install -q pyscipopt
! pip install pandas
! pip install gurobipy

In [ ]:
import pandas as pd
import math
from gurobipy import Model, GRB, quicksum

# Optimierungsmodell zur Elektrifizierung der Logistik

### Import der CSV Dateien

In [ ]:
import os

# Prüfen ob in Colab
try:
    import google.colab
    IN_COLAB = True
except:
    IN_COLAB = False

if IN_COLAB:
    # In Colab: Repository klonen (nur einmal)
    if not os.path.exists('Fallstudie_SCM'):
        !git clone https://github.com/Olfeng-xalaz/Fallstudie_SCM.git
    folder = "Fallstudie_SCM/DataCSV"
else:
    # Lokal in VS Code: Relativer Pfad
    folder = os.path.join(os.path.dirname(__file__), "DataCSV") if "__file__" in dir() else "DataCSV"

In [ ]:
chargers = pd.read_csv(f"{folder}/chargers.csv", sep=";")

In [ ]:
chargers.head()

In [ ]:
dtrucks_specs = pd.read_csv(f"{folder}/diesel_trucks.csv", sep=";")

In [ ]:
dtrucks_specs.head()

In [ ]:
etrucks_specs = pd.read_csv(f"{folder}/electric_trucks.csv", sep=";")

In [ ]:
etrucks_specs.head()

In [ ]:
routes = pd.read_csv(f"{folder}/routes.csv", sep=";")

In [ ]:
routes

### Indexmengen

In [ ]:
#Erstellen einer Modellinstanz
m = Model("Electrification")

In [ ]:
R = routes["route_id"].unique() # Menge der Routen
C = chargers["charger_model"].unique() # Menge der Charger-Modelle
L = pd.concat([dtrucks_specs["truck_model"],etrucks_specs["truck_model"]]).unique() # Menge der Fahrzeugmodelle
P = range(1, 21) # Potenzielle Fahrzeuge
T = range(0, 96) # 96 Zeitintervalle pro Tag
I = range(1, 4) # Potenzielle Säulen-Slots


print("R (Routen):", R)
print("C (Charger):", C)
print("L (Fahrzeugmodelle):", L)
print("P (potenzielle Fahrzeuge):", list(P))
print("T (Zeitintervalle):", list(T))
print("I (Säulen):", list(I))

### Parameter

In [ ]:
# 3) Parameter aus routes.csv
# -----------------------------
dist_total = dict(zip(routes["route_id"], routes["distance_total"]))
dist_toll  = dict(zip(routes["route_id"], routes["distance_toll"]))
start_time = dict(zip(routes["route_id"], routes["starttime"]))
end_time   = dict(zip(routes["route_id"], routes["endtime"]))

# Hinweis:
# A[r,t] (binär: Tour r läuft im Intervall t) und g[r,t] (kWh-Verbrauch pro Intervall)
# hängen von Zeitdiskretisierung + Tourdauer ab.
# Das bauen wir später, sobald klar ist, wie starttime/endtime formatiert sind.

# -----------------------------
# 4) Parameter aus chargers.csv
# -----------------------------
capex_ch = dict(zip(chargers["charger_model"], chargers["capex_yearly"]))
opex_ch  = dict(zip(chargers["charger_model"], chargers["opex_yearly"]))
pmax_ch  = dict(zip(chargers["charger_model"], chargers["max_power"]))        # kW
spots_ch = dict(zip(chargers["charger_model"], chargers["charging_spots"]))   # Anzahl Ladepunkte

# -----------------------------
# 5) Parameter aus electric_trucks.csv
# -----------------------------
capex_veh = {}
opex_veh  = {}

cons_e = {}       # kWh/100km
thg_e  = {}       # €/a
pmax_veh = {}     # kW max Ladeleistung
batt_kwh = {}     # kWh Batterie (soc_max)

for _, row in etrucks_specs.iterrows():
    truck_model = row["truck_model"]
    capex_veh[truck_model] = row["capex_yearly"]
    opex_veh[truck_model]  = row["opex_yearly"]
    cons_e[truck_model]    = row["avg_energy_kWh_per_100km"]
    thg_e[truck_model]     = row["thg_yearly"]
    pmax_veh[truck_model]  = row["max_power"]
    batt_kwh[truck_model]  = row["soc_max_kWh"]

batt_kwh['ActrosL'] = 0
pmax_veh['ActrosL'] = 0

# -----------------------------
# 6) Parameter aus diesel_trucks.csv
# -----------------------------
kfz_d = {}       # €/a
cons_d = {}      # ggf. l/100km (nur falls du Diesel-Kraftstoffkosten modellierst)

for _, row in dtrucks_specs.iterrows():
    truck_model = row["truck_model"]
    capex_veh[truck_model] = row["capex_yearly"]
    opex_veh[truck_model]  = row["opex_yearly"]
    if "kfz_yearly" in dtrucks_specs.columns:
        kfz_d[truck_model] = row["kfz_yearly"]
    if "avg_diesel_per_100km" in dtrucks_specs.columns:
        cons_d[truck_model] = row["avg_diesel_per_100km"]

cons_e['ActrosL'] = 0
print(cons_e)

# -----------------------------
# 7) Abgeleitete Parameter: Energiebedarf pro Route und e-Lkw (E[r,e])
# -----------------------------
E_route_e = {}  # (r,e) -> kWh

for r in R:
    for e in cons_e.keys():
        E_route_e[(r, e)] = dist_total[r] * cons_e[e] / 100.0

# -----------------------------
# 8) Falltext-Parameter (Konstanten)
# -----------------------------
N_days = 260
delta_h = 0.25  # 15 Minuten = 0.25 Stunden
diesel = 1.60
SOC_T_Start = 0

# Netz & Tarif
P_grid_max = 500.0 # max kW am Depot
P_grid_add = 500.0 # Zusatzleistung bei Netzausbau
capex_grid_add = 10000.0 # Kosten/Jahr für Netzausbau

c_energy = 0.25   # Arbeitspreis Strom in €/kWh
c_capex = 1000.0   # Stromkosten in €/a
c_peak = 150.0    # Leistungspreis in €/kW

# Maut
c_toll = 0.34      # €/km mautpflichtig

# Speicher
c_capex_bat_kW = 30.0      # Batteriekosten in €/kW
c_capex_bat_kWh = 350.0     # Batteriekosten in €/kWh
roundtrip_eff = 0.98 # Round-Trip Efficiency aka Wirkungsgrad
eta = math.sqrt(roundtrip_eff)  # für lineare Lade/Entlade-Gleichungen
dod = 0.975 # Max. Entladetiefe
soc_bat_min_frac = 1.0 - dod    # = 0.025


# -----------------------------
# 9) Kurzer Test-Print der wichtigsten Parameter
# -----------------------------
print("Beispiel dist_total[r]:", list(dist_total.items())[:3]); print()
print("Charger pmax (kW):", pmax_ch); print()
print("e-Lkw batt_kwh:", batt_kwh); print()
print("Energiebedarf Beispiel (erste Route, e400/e600):")
first_r = R[0]
for e in cons_e.keys():
    print(" ", (first_r, e), "=", E_route_e[(first_r, e)], "kWh")
print(E_route_e)

print("Konstanten: P_grid_max =", P_grid_max, "| c_energy =", c_energy, "| c_toll =", c_toll); print()
print("Speicher: eta =", eta, "| soc_bat_min_frac =", soc_bat_min_frac); print()

## Hilfsfunktionen

In [ ]:
def time_to_t_interval(uhrzeit):
  # Den String am Doppelpunkt trennen
  stunden_str, minuten_str = uhrzeit.split(":")
  # In Zahlen (Integer) umwandeln
  stunden = int(stunden_str)
  minuten = int(minuten_str)
  t = stunden * 4
  t = t + minuten / 15
  return t

## Weitere Parameter

In [ ]:
# Verbrauch pro Zeitintervall pro LKW Art
# Erweitere deine Verbrauchsmatrix
verbrauch_pro_intervall = {}
for r in R:
    # 1. Daten holen (aus deinen Routen-Daten)
    dist = dist_total[r]
    start = time_to_t_interval(start_time[r])
    ende = time_to_t_interval(end_time[r])
    dauer = ende - start

    for l in L:
        # 2. Verbrauch pro Intervall berechnen
        if l in ['eActros400', 'eActros600']:
            # Elektro: Distanz * Verbrauch / Dauer
            #print(dist, cons_e[l], dauer)
            kwh_pro_t = (dist/100 * cons_e[l]) / dauer
        else:
            # Diesel: Verbraucht 0 kWh Strom
            kwh_pro_t = 0

        # 3. In Matrix speichern
        for t in T:
            if start <= t < ende:
                verbrauch_pro_intervall[r, l, t] = kwh_pro_t
            else:
                verbrauch_pro_intervall[r, l, t] = 0

In [ ]:
route_aktiv = {}
for r in R:
    start_t = time_to_t_interval(start_time[r])
    ende_t = time_to_t_interval(end_time[r])
    for t in T:
        if start_t <= t < ende_t:
            route_aktiv[r, t] = 1
        else:
            route_aktiv[r, t] = 0

## Entscheidungsvariablen

In [ ]:
# =============================================================================
# 1. ENTSCHEIDUNGSVARIABLEN (DECISION VARIABLES)
# =============================================================================
# Definition der Variablen, die der Gurobi-Solver optimieren soll.
# Ziel ist es, die kostengünstigste Kombination aus Fahrzeugen, Infrastruktur
# und Ladeplänen zu finden.

# --- A. Strategische Flottenentscheidung ---
# Entscheidung: Welches Fahrzeugmodell (l) wird welcher physischen ID (p) zugewiesen?
# Relevanz für Fallstudie: Dies ist der Kern der TCO-Berechnung. Hier entscheidet
# sich, ob ein teurer E-LKW (mit niedrigen Betriebskosten) oder ein günstiger
# Diesel (mit hohen Betriebskosten) angeschafft wird.
Flottenwahl = {}
for l in L:
    for p in P:
        Flottenwahl[l, p] = m.addVar(vtype = GRB.BINARY, name =f"LKW_{l}_auf_ID_{p}")

# --- B. Operative Routenzuordnung ---
# Entscheidung: Welcher LKW (p) fährt welche Route (r)?
# Relevanz: Bestimmt den täglichen Energiebedarf. Ein E-LKW kann nur Routen
# fahren, die seine Reichweite (unter Berücksichtigung von Nachladen) zulässt.
Zuordnung_LKW_Route = {}
for r in R:
    for p in P:
        Zuordnung_LKW_Route[r, p] = m.addVar(vtype = GRB.BINARY, name =f"Route_{r}_wird_von_LKW_mit_ID_{p}_gefahren")

# --- C. Infrastruktur-Aufbau (Ladehof) ---
# Entscheidung: Welcher Chargertyp (c) wird an welchem Ladeplatz (i) installiert?
# Relevanz: Bestimmt die Investitionskosten (Capex) der Infrastruktur.
# Der Solver kann auch entscheiden, Plätze leer zu lassen, um Kosten zu sparen.
Auswahl_Charger = {}
for c in C:
    for i in I:
        Auswahl_Charger[c, i] = m.addVar(vtype = GRB.BINARY, name =f"Charger_{c}_ist_Säule_Nummer_{i}")

# --- D. Lade-Zeitplan (Smart Charging) ---
# Entscheidung: Steht LKW (p) zum Zeitpunkt (t) an Ladesäule (i)?
# Relevanz: Dies ist die komplexeste Variable. Sie muss das "Nacht-Umparkverbot"
# (Blockieren der Säule von 18:00-06:00 Uhr) sowie die Verfügbarkeit der LKWs abbilden.
Zuordnung_LKW_Zeitpunkt_Charger = {}
for p in P:
    for t in T:
        for i in I:
            Zuordnung_LKW_Zeitpunkt_Charger[p,t,i] = m.addVar(vtype = GRB.BINARY, name =f"LKW_{p}_zum_Zeitpunkt_{t}_an_Charger_{i}")

# --- E. Fahrzeug-Zustand & Energiefluss ---
# SOC_Zeit_Fahrzeug: Batterie-Füllstand in kWh. Muss Restriktionen (Min/Max) einhalten.
# Ladeleistung: Tatsächliche Leistung in kW, die aus dem Netz gezogen wird.
# Dient zur Berechnung der Stromkosten und Überwachung des Netzanschlusses.
SOC_Zeit_Fahrzeug = {}
Ladeleistung = {}
for p in P:
  for t in T:
    SOC_Zeit_Fahrzeug[p,t] = m.addVar(lb=0, name=f"SOC_FahrzeugID_{p}_Zeitpunkt{t}")
    Ladeleistung[p, t] = m.addVar(lb=0, name=f"Ladeleistung_{p}_{t}")

# --- F. Stationärer Batteriespeicher (Peak Shaving) ---
# Entscheidung: Wie wird der Speicher betrieben, um Lastspitzen zu kappen?
# SOC_Speicher: Füllstand des stationären Speichers.
# Laden/Entladen: Energiefluss zur Netzunterstützung oder Depotversorgung.
SOC_Speicher = {}      # Aktueller Stand in kWh
Laden_Speicher = {}    # Leistung vom Netz in den Speicher (kW)
Entladen_Speicher = {} # Leistung vom Speicher ins Depot/Lkw (kW)

for t in T:
    SOC_Speicher[t] = m.addVar(lb=0, name=f"SOC_Speicher_t{t}")
    Laden_Speicher[t] = m.addVar(lb=0, name=f"Laden_Speicher_t{t}")
    Entladen_Speicher[t] = m.addVar(lb=0, name=f"Entladen_Speicher_t{t}")

# --- G. Hilfsvariable zur Linearisierung (Kosten-Mapping) ---
# Diese Variable verknüpft Route, Modell und ID eindeutig (wird 1, wenn alles zutrifft).
# Notwendig, um in der Zielfunktion modellspezifische Kosten (z.B. Dieselverbrauch vs. Stromverbrauch)
# korrekt auf die gefahrene Route zu mappen.
Einsatz = {}
for r in R:
    for l in L:
        for p in P:
            Einsatz[r, l, p] = m.addVar(vtype=GRB.BINARY, name=f"Einsatz_{r}_{l}_{p}")

# --- H. Netzinfrastruktur & Limits ---
# Entscheidung: Soll der Netzanschluss kostenpflichtig erweitert werden?
# Wenn 1, erhöht sich das Limit am Anschlusspunkt (GCP).
Ausbau_Netz = m.addVar(vtype=GRB.BINARY, name="Netzausbau")

# Dimensionierung des Batteriespeichers (Investitionsentscheidung)
# Der Solver bestimmt die optimale Größe, um Peak-Kosten gegen Invest-Kosten abzuwägen.
Batterie_kWh = m.addVar(lb=0, name="Batteriekapa_kWh")
Batterie_kW = m.addVar(lb=0, name="Batterieleistung_kW")

# Variable zur Erfassung der höchsten Lastspitze des Jahres (für Leistungspreis-Berechnung).
P_peak = m.addVar(lb=0, name="Jahres_Leistungsspitze_kW")

# --- I. Technische Hilfsvariable (Säulen-Limit) ---
# Dient der korrekten Verteilung der Ladeleistung auf die spezifischen Ladesäulen.
# Stellt sicher, dass ein LKW an einer 150kW-Säule nicht mit 300kW lädt.
Ladeleistung_Saeule = {}
for p in P:
    for t in T:
        for i in I:
            Ladeleistung_Saeule[p, t, i] = m.addVar(lb=0, ub=500, name=f"P_S_{p}_{t}_{i}")

m.update()
len(m.getVars())

## Restriktionen

In [ ]:
# =============================================================================
# 2. RESTRIKTIONEN (CONSTRAINTS)
# =============================================================================
# Hier wird das mathematische Regelwerk definiert. Diese Nebenbedingungen stellen
# sicher, dass die gefundene Lösung physikalisch möglich und betrieblich sinnvoll ist.

# Hilfswert für Big-M Restriktionen (Beschleunigung des Solvers):
# Wir nutzen die maximale Ladeleistung des stärksten Fahrzeugs als Obergrenze.
M_power = max(pmax_veh.values())

# --- A. Logische Verknüpfung der Entscheidungsebenen ---
# Das Modell muss sicherstellen, dass Route, Fahrzeugmodell und physischer LKW
# konsistent zueinander passen. Nur so können wir die Kosten (Diesel vs. Strom)
# der richtigen Route zuordnen.

#Für Einsatz{}
for r in R:
    for l in L:
        for p in P:
            # 1. Konsistenzprüfung Route: Wenn Variable 'Einsatz' 1 ist, muss auch
            # die Zuordnung von LKW p zu Route r aktiv sein.
            m.addConstr(Einsatz[r, l, p] <= Zuordnung_LKW_Route[r, p])

            # 2. Konsistenzprüfung Modell: Wenn Variable 'Einsatz' 1 ist, muss
            # LKW p auch das Modell l besitzen (Flottenwahl).
            m.addConstr(Einsatz[r, l, p] <= Flottenwahl[l, p])

            # 3. Erzwingung: Wenn LKW p die Route r fährt UND Modell l hat,
            # DANN MUSS die Einsatz-Variable zwingend 1 sein.
            # (Dies ist notwendig für die korrekte Kostenberechnung in der Zielfunktion).
            m.addConstr(Einsatz[r, l, p] >= Zuordnung_LKW_Route[r, p] + Flottenwahl[l, p] - 1)
m.update()


# --- B. Routenplanung (VRP - Vehicle Routing Problem) ---
# Sicherstellung, dass alle logistischen Aufträge erfüllt werden.

# Zuordnung_LKW_Route{}
#1. Erfüllungspflicht: Jede definierte Route r muss von exakt einem LKW bedient werden.
for r in R:
  m.addConstr(quicksum(Zuordnung_LKW_Route[r,p] for p in P)==1)


#2. Modell-Pflicht: Ein LKW kann eine Route nur fahren, wenn ihm auch ein
# physisches Modell (Diesel oder Elektro) zugewiesen wurde.
# Verhindert "Geisterfahrten" ohne zugewiesenes Fahrzeug.
for r in R:
    for p in P:
        # Wenn Route r dem Fahrzeug p zugeordnet ist (Variable == 1),
        # dann muss die Summe der Flottenwahl über alle Modelle l für dieses p ebenfalls 1 sein.
        m.addConstr(
            Zuordnung_LKW_Route[r, p] <= quicksum(Flottenwahl[l, p] for l in L),
            name=f"Route_braucht_Modell_{r}_{p}"
        )

m.update()

# --- C. Flotten-Restriktionen ---
# Strategische Grenzen für die Anschaffung.

# Flottenwahl{}
#1. Fuhrpark-Obergrenze: Es dürfen maximal 20 Fahrzeuge beschafft werden.
m.addConstr(quicksum(Flottenwahl[l,p] for p in P for l in L)<=20)

#2. Eindeutigkeit: Jede physische Fahrzeug-ID (p) darf maximal ein Modell (l) besitzen.
# Ein LKW kann nicht gleichzeitig ein Diesel und ein E-LKW sein.
for p in P:
  m.addConstr(quicksum(Flottenwahl[l,p] for l in L)<=1)

m.update()

# --- D. Batterie-Management (SOC) ---
# Bilanzierung der Energieströme im Fahrzeugspeicher.

# SOC_Zeit_Fahrzeug{}
#1. Berechnung des Ladezustands (SOC) für jeden Zeitschritt t.
for p in P:
    for t in T:
        if t == 0:
            # Startbedingung: Definierter SOC zu Beginn des Tages (00:00 Uhr).
            m.addConstr(SOC_Zeit_Fahrzeug[p, t] == SOC_T_Start, name=f"Start_SOC_{p}")
        else:
            # Energiebilanz: SOC(t) = SOC(t-1) + Laden - Verbrauch
            t_prev = t - 1
            # Umrechnung Leistung (kW) in Arbeit (kWh) für 15-Minuten-Intervalle (Faktor 0.25)
            energie_geladen = Ladeleistung[p, t] * 0.25
            energie_verbrauch = quicksum(Einsatz[r, l, p] * verbrauch_pro_intervall[r, l, t] for r in R for l in L)

            m.addConstr(SOC_Zeit_Fahrzeug[p, t] == SOC_Zeit_Fahrzeug[p, t_prev] + energie_geladen - energie_verbrauch,
                         name=f"SOC_Update_{p}_{t}")

#2. Zyklische Randbedingung: Der SOC am Ende des Tages muss dem Startwert entsprechen.
# Dies verhindert, dass die Flotte leergefahren wird und am nächsten Tag nicht einsatzbereit ist.
for p in P:
  m.addConstr(SOC_Zeit_Fahrzeug[p,0] == SOC_Zeit_Fahrzeug[p, 95])

#3. Kapazitätsgrenze: Der SOC darf die Batteriegröße des gewählten Modells nicht überschreiten.
# Dies ist variabel, da verschiedene LKW-Modelle unterschiedliche Akkugrößen haben.
for p in P:
    for t in T:
        # Der SOC von LKW p zum Zeitpunkt t darf die Kapazität
        # des gewählten Modells l nicht überschreiten.
        m.addConstr(
            SOC_Zeit_Fahrzeug[p, t] <= quicksum(Flottenwahl[l, p] * batt_kwh[l] for l in L),
            name=f"Max_Kapazitaet_LKW_{p}_t{t}"
        )

# 4. SOC niemals < 0 bereits über lower bound definiert


# --- E. Operative Fahrpläne ---
# Sicherstellung, dass ein LKW nicht an zwei Orten gleichzeitig sein kann.

# Route_Aktiv{}
# 1. Keine_Ueberlappung: Ein LKW kann zu einem Zeitpunkt t maximal auf einer Route aktiv sein.
for p in P:
    for t in T:
        # Die Summe aller Routen r, die zum Zeitpunkt t aktiv sind,
        # darf für LKW p nicht größer als 1 sein.
        m.addConstr(
            quicksum(Zuordnung_LKW_Route[r, p] * route_aktiv[r, t] for r in R) <= 1,
            name=f"Keine_Ueberlappung_LKW_{p}_t{t}"
        )
m.update()

# --- F. Smart Charging Logik ---
# Regeln für das Laden: Wann, wo und wie viel?

# Ladeleistung [p,t]
# 1. Ladeverbot auf Tour: Ein LKW kann nur laden, wenn er sich im Depot befindet.
for p in P:
    for t in T:
        # Summe ist 1, wenn der LKW fährt, 0 wenn er im Depot ist
        ist_auf_tour = quicksum(Zuordnung_LKW_Route[r, p] * route_aktiv[r, t] for r in R)

        # Wenn ist_auf_tour == 1, muss Ladeleistung <= 0 sein (also 0)
        # Wenn ist_auf_tour == 0, darf er laden (begrenzt durch max. Fahrzeugleistung)
        m.addConstr(Ladeleistung[p, t] <= (1 - ist_auf_tour) * max(pmax_veh.values()),
                     name=f"Laden_nur_im_Depot_{p}_{t}")

# 3. Fahrzeug-Limit: Die Ladeleistung ist physikalisch durch das BMS des LKWs begrenzt.
for p in P:
    for t in T:
        # Die Ladeleistung darf die fahrzeugspezifische Grenze nicht überschreiten
        m.addConstr(
            Ladeleistung[p, t] <= quicksum(Flottenwahl[l, p] * pmax_veh[l] for l in L),
            name=f"Max_Ladeleistung_Fahrzeug_{p}_{t}"
        )
# --- KORREKTUR: Ladeleistung & Säulenkapazität ---
# Detaillierte Zuordnung der Leistung auf spezifische Säulenhardware.

for p in P:
    for t in T:
        # A. Leistungsbilanz: Die Summe der Leistung aller Ladevorgänge an Säulen
        # muss der Gesamtleistung entsprechen, die der LKW aus dem Netz zieht.
        m.addConstr(
            quicksum(Ladeleistung_Saeule[p, t, i] for i in I) == Ladeleistung[p, t],
            name=f"Leistungsbilanz_{p}_{t}"
        )

        for i in I:
            # B. Physische Kopplung: Strom kann nur fließen, wenn der Stecker
            # (Zuordnung_LKW_Zeitpunkt_Charger) auch tatsächlich in der Säule steckt.
            m.addConstr(
                Ladeleistung_Saeule[p, t, i] <= max(pmax_veh.values()) * Zuordnung_LKW_Zeitpunkt_Charger[p, t, i],
                name=f"Leistungs_Kopplung_{p}_{t}_{i}"
            )

# C. Hardware-Limit Säule: Die Summe der Leistung aller Fahrzeuge an einer Säule
# darf die Nennleistung dieser Säule (z.B. 300 kW Hypercharger) nicht überschreiten.
for i in I:
    for t in T:
        m.addConstr(
            quicksum(Ladeleistung_Saeule[p, t, i] for p in P) <=
            quicksum(Auswahl_Charger[c, i] * pmax_ch[c] for c in C),
            name=f"Echte_Saeulenkapazitaet_i{i}_t{t}"
        )

# D. Netzanschluss & Rückspeiseschutz
for t in T:
    # Berechnung der Gesamtlast am Netzanschlusspunkt (GCP): LKWs + Speicher
    netzbezug = quicksum(Ladeleistung[p, t] for p in P) + Laden_Speicher[t] - Entladen_Speicher[t]

    # 1. Limitierung durch Transformator (500 kW Basis + optionale Erweiterung)
    m.addConstr(netzbezug <= 500 + (500 * Ausbau_Netz), name=f"Netz_Limit_t{t}")

    # 2. Verbot der Rückspeisung ins öffentliche Netz (Vorgabe Netzbetreiber)
    m.addConstr(netzbezug >= 0, name=f"Keine_Rückspeisung_t{t}")

m.update()

# --- G. Belegungsplanung der Ladeplätze ---

#Zuordnung_LKW_Zeitpunkt_Charger[p,t,i]
#1. Stecker-Limit: An einer Säule dürfen nur so viele LKWs hängen, wie sie Stecker hat.
# (z.B. Hypercharger hat 2 Plätze, AC-Wallbox oft nur 1).
for i in I:
    for t in T:
        m.addConstr(
            quicksum(Zuordnung_LKW_Zeitpunkt_Charger[p, t, i] for p in P)
            <= quicksum(spots_ch[c] * Auswahl_Charger[c, i] for c in C),
            name=f"Spots_Limit_Saeule_{i}_t{t}"
        )
#2. Eindeutigkeit LKW: Ein Fahrzeug kann physikalisch nur an maximal einer Säule gleichzeitig laden.
for p in P:
    for t in T:
        # Ein LKW p kann zum Zeitpunkt t an maximal einer Säule i hängen
        m.addConstr(
            quicksum(Zuordnung_LKW_Zeitpunkt_Charger[p, t, i] for i in I) <= 1,
            name=f"Max_Eine_Saeule_Pro_LKW_{p}_{t}"
        )

# 3. Lade-Kontinuität ("Stickiness"): Kein "Säulen-Hopping".
# Wenn ein LKW über mehrere Intervalle lädt, darf er nicht zwischendurch die Säule wechseln.
for p in P:
    for t in T:
        if t < 95: # Nicht für das letzte Intervall (t+1 wäre sonst out of range)
            for i in I:
                # Logik: Wenn (LKW an Säule i zu t) UND (LKW lädt zu t+1 irgendwo anders),
                # dann ist das verboten.
                m.addConstr(
                    Zuordnung_LKW_Zeitpunkt_Charger[p, t, i]
                    + quicksum(Zuordnung_LKW_Zeitpunkt_Charger[p, t+1, j] for j in I if j != i)
                    <= 1,
                    name=f"Kein_Saeulenwechsel_{p}_{t}_{i}"
                )

# 4. Nacht-Umparkverbot (Kern der Fallstudie)
# Realitäts-Check: Zwischen 18:00 und 06:00 Uhr ist kein Personal vor Ort, um LKWs umzuparken.
# Wer abends an einer Säule steht, blockiert diese zwingend bis zum Morgen (oder Abfahrt).
T_Nacht = [t for t in T if t >= 72 or t < 24]

for p in P:
    for t in T_Nacht:
        t_next = (t + 1) % 96
        # Nur wenn das nächste Intervall auch noch in der Nacht liegt
        if t_next in T_Nacht:
            # Wenn der Lkw NICHT auf Tour ist (also im Depot parkt)
            ist_im_depot = 1 - quicksum(Zuordnung_LKW_Route[r, p] * route_aktiv[r, t] for r in R)

            for i in I:
                # Regel: Wenn er an Säule i steht, muss er dort stehen bleiben,
                # solange er im Depot ist (kein Umparken nachts).
                # Logik: Wenn er zu t angeschlossen war und zu t+1 noch da ist, MUSS er angeschlossen bleiben.
                m.addConstr(
                    Zuordnung_LKW_Zeitpunkt_Charger[p, t, i]
                    + (1 - quicksum(Zuordnung_LKW_Route[r, p] * route_aktiv[r, t_next] for r in R))
                    - Zuordnung_LKW_Zeitpunkt_Charger[p, t_next, i] <= 1,
                    name=f"Nacht_Anschluss_Pflicht_{p}_{t}_{i}"
                )

m.update()

# --- H. Infrastruktur-Auswahl ---
# Physische Einschränkung der Ladeplätze.

#Auswahl_Charger(c,i)
#1. Einzigartigkeit: An jedem Ladeplatz i darf maximal ein Hardware-Typ c installiert werden.
for i in I:
    # Jede Säule i (1, 2, 3) darf maximal einen Typ c haben
    m.addConstr(
        quicksum(Auswahl_Charger[c, i] for c in C) <= 1,
        name=f"Max_Ein_Typ_Pro_Saeule_{i}"
    )

m.update()

# --- I. Stationärer Batteriespeicher ---
# Dient dem Peak-Shaving (Lastspitzenkappung) und der Netzentlastung.

#Batteriespeicher
# 1. SOC-Berechnung (Energiebilanz Speicher)
for t in T:
    t_prev = t - 1 if t > 0 else 95

    # Bilanz: Stand vorher + Laden (mit Verlust) - Entladen (mit Verlust)
    # Faktor 0.99 repräsentiert Wirkungsgradverluste.
    m.addConstr(
        SOC_Speicher[t] == SOC_Speicher[t_prev]
        + (Laden_Speicher[t] * 0.25 * 0.99)
        - (Entladen_Speicher[t] * 0.25 / 0.99),
        name=f"SOC_Bilanz_Speicher_t{t}"
    )

for t in T:
    # 1. Maximale Kapazität: Speicherinhalt darf installierte Größe nicht überschreiten.
    m.addConstr(SOC_Speicher[t] <= Batterie_kWh, name=f"Speicher_Max_Kappa_t{t}")

    # 2. Tiefentladeschutz: Speicher darf nicht unter 2.5% fallen (Lebensdauer).
    m.addConstr(SOC_Speicher[t] >= 0.025 * Batterie_kWh, name=f"Speicher_Min_DoD_t{t}")

for t in T:
    # Leistungslimits für Laden/Entladen basierend auf installierter Wechselrichter-Leistung.
    m.addConstr(Laden_Speicher[t] <= Batterie_kW, name=f"Speicher_Max_Laden_t{t}")
    m.addConstr(Entladen_Speicher[t] <= Batterie_kW, name=f"Speicher_Max_Entladen_t{t}")

# Zyklus-Bedingung: Speicherstand am Ende = Speicherstand am Anfang (Nachhaltigkeit).
m.addConstr(SOC_Speicher[0] == SOC_Speicher[95], name="Speicher_Zyklus_Check")


# --- J. Peak Shaving (Lastspitzen-Management) ---

# Peak Leistung
# Ermittlung der Jahreshöchstlast für die Abrechnung des Leistungspreises.
# P_peak muss größer/gleich sein als die Netzlast in JEDEM Zeitintervall.
for t in T:
    # Berechne die tatsächliche Netzlast in diesem Intervall:
    # Last = (Summe aller LKW-Ladeleistungen) + (Speicher laden) - (Speicher entladen)
    aktuelle_netzlast = quicksum(Ladeleistung[p, t] for p in P) + Laden_Speicher[t] - Entladen_Speicher[t]

    # Der Solver wird P_peak so klein wie möglich halten (wegen Kosten), aber er
    # muss mindestens so groß sein wie die höchste Lastspitze.
    m.addConstr(P_peak >= aktuelle_netzlast, name=f"Peak_Check_t{t}")

m.update()


# --- K. Solver-Performance & Symmetrie-Brechung ---
# Diese Restriktionen ändern nicht das Ergebnis, beschleunigen aber die Rechnung drastisch,
# indem sie mathematisch identische Lösungen ausschließen.

# Rechenzeit optimierung
# 1. Fahrzeug-Symmetrie: Sortiert die Fahrzeuge. Verhindert, dass der Solver ID 1 und ID 2 vertauscht.
for i in range(len(P) - 1):
    p_current = P[i]
    p_next = P[i+1]
    # "Ein Modell für Fahrzeug i+1 darf nur gewählt werden, wenn auch für Fahrzeug i eines gewählt wurde"
    m.addConstr(
        quicksum(Flottenwahl[l, p_current] for l in L) >=
        quicksum(Flottenwahl[l, p_next] for l in L),
        name=f"Symmetry_Break_{p_current}_{p_next}"
    )

# 2. Symmetrie-Brechung Ladesäulen:
# Eine Säule i+1 darf nur gebaut werden, wenn Säule i auch gebaut ist.
# Das verhindert Lücken wie [Gebaut, Leer, Gebaut] und zwingt den Solver, vorne anzufangen.
for i in range(len(I) - 1):
    m.addConstr(
        quicksum(Auswahl_Charger[c, I[i]] for c in C) >=
        quicksum(Auswahl_Charger[c, I[i+1]] for c in C),
        name=f"Charger_Order_{i}"
    )

m.update()
print(m.NumConstrs)

## Zielfunktion

In [ ]:
# =============================================================================
# 3. ZIELFUNKTION (OBJECTIVE FUNCTION)
# =============================================================================
# Ziel: Minimierung der Total Cost of Ownership (TCO) pro Tag.
# Alle Investitionskosten (CAPEX) und jährlichen Fixkosten (OPEX) werden auf
# einen Betriebstag heruntergebrochen (durch 260 Arbeitstage geteilt), um sie
# mit den variablen Tageskosten (Strom, Diesel, Maut) vergleichbar zu machen.

# 1. Anteilige Fahrzeugkosten (Tägliche Abschreibung + Fixkosten)
# Hier zeigt sich der TCO-Effekt: E-LKWs haben hohe Capex, aber durch die
# THG-Quote (Treibhausgasminderungsquote) verringern sich die effektiven Kosten.
obj_veh = (1/260) * quicksum(
    Flottenwahl[l, p] * (capex_veh[l] + opex_veh[l] - thg_e.get(l, 0))
    for l in L for p in P
)

# 2. Anteilige Kosten Ladeinfrastruktur (Capex + Opex)
# Setzt sich zusammen aus allgemeinen Fixkosten für den Netzanschluss (c_capex)
# und den spezifischen Kosten pro installierter Ladesäule (Hardware + Installation).
# Der Solver wägt ab: Wenige teure Hypercharger vs. viele günstige AC-Lader.
obj_charger = (1/260) * (c_capex +
    quicksum(Auswahl_Charger[c, i] * (capex_ch[c] + opex_ch[c]) for c in C for i in I)
)

# 3. Anteilige Kosten Stationärer Batteriespeicher
# Investition in Flexibilität: Der Speicher verursacht Kosten für Kapazität (kWh)
# und Leistungselektronik (kW) sowie jährliche Wartung (hier pauschal 2% der Investition).
# Diese Kosten lohnen sich nur, wenn dadurch teure Lastspitzen (Peak Shaving) verhindert werden.
obj_battery = (1/260) * (
    (Batterie_kWh * c_capex_bat_kWh) +
    (Batterie_kW * c_capex_bat_kW) +
    (Batterie_kWh * 0.02 * c_capex_bat_kWh) +
    (Batterie_kW * 0.02 * c_capex_bat_kW)
)

# 4. Anteilige Kosten Netzausbau (Baukostenzuschuss)
# Einmalige Kosten, falls das bestehende Trafo-Limit (500 kW) nicht ausreicht.
# Der Solver versucht meist, diese hohen Sprungkosten durch intelligentes Laden zu vermeiden.
obj_grid = (1/260) * (Ausbau_Netz * capex_grid_add)

# 5. Variable Kosten (Laufende Betriebskosten pro Tag)

# Stromkosten (Energiepreis)
# Berechnung der tatsächlichen Strommenge in kWh (Leistung in kW * 0.25h).
# Berücksichtigt Laden der LKWs sowie Speicherverluste (Laden/Entladen).
cost_electricity = quicksum(
    (quicksum(Ladeleistung[p, t] for p in P) + Laden_Speicher[t] - Entladen_Speicher[t]) *0.25 * c_energy
    for t in T
)

# Leistungspreis (Peak Shaving)
# Gebühr für die höchste im Jahr auftretende Lastspitze (kW).
# Da P_peak die Jahresspitze ist, legen wir diese Kosten anteilig auf den Tag um.
cost_peak = (1 / 260) * P_peak * c_peak

# Mautkosten (LKW-Maut)
# Wichtiger Hebel in der Fallstudie: Diesel-LKWs ('ActrosL') zahlen volle Maut.
# E-LKWs sind oft befreit oder zahlen reduziert (wird über if-Abfrage gesteuert).
cost_toll = quicksum(
    Einsatz[r, 'ActrosL', p] * dist_toll[r] * c_toll
    for r in R for p in P if 'ActrosL' in L
)

# Treibstoffkosten (Diesel)
# Nur relevant für Verbrenner. Berechnung basierend auf Distanz und Durchschnittsverbrauch.
cost_diesel = quicksum(
    Einsatz[r, 'ActrosL', p] * (dist_total[r] * cons_d.get('ActrosL', 0) / 100.0) * diesel
    for r in R for p in P if 'ActrosL' in L
)

# 6. KFZ-Steuer (Fahrzeugsteuer)
# Spezifische Fixkosten für Diesel-Fahrzeuge. E-LKWs sind oft steuerbefreit.
# Wir wandeln den Jahresbetrag in einen Tageswert um.
cost_tax = (1/260) * quicksum(
    Flottenwahl['ActrosL', p] * kfz_d['ActrosL']
    for p in P
)

# GESAMT-ZIELFUNKTION
# Der Solver minimiert die Summe aller oben genannten Kostenkomponenten.
m.setObjective(
    obj_veh + obj_charger + obj_battery + obj_grid + cost_electricity + cost_peak + cost_toll + cost_diesel + cost_tax,
    GRB.MINIMIZE
)

## Berechnung des Ergebnisses

In [ ]:
#m.setRealParam("limits/time", 30)
m.optimize()

In [ ]:
print ("Zielfunktions Wert Kosten: ",m.objVal)

## Solution Output

In [ ]:
print("--- Fahrzeugmodell-Zuordnung ---")
for l in L:
    for p in P:
        if Flottenwahl[l, p].X > 0.5:
            print(f"Fahrzeug ID {p}: Modell {l}")


print("\n--- Optimierte Ladeergebnisse ---")
for t in T:
    # Checke, ob in diesem Intervall überhaupt etwas passiert
    if any(Ladeleistung[p, t].X > 0.1 for p in P):
        print(f"Zeitintervall {t}:")
        for p in P:
            p_val = Ladeleistung[p, t].X
            if p_val > 0.1:
                # Finde die Säule
                saeule = next((i for i in I if Zuordnung_LKW_Zeitpunkt_Charger[p, t, i].X > 0.5), "Keine")
                print(f"  - LKW {p} lädt {p_val:.1f} kW an Säule {saeule}")

print("\n--- Routen-Zuordnung (Route Assignment) ---")
for r in R:
    for p in P:
        if Zuordnung_LKW_Route[r, p].X > 0.5:
            print(f"Route {r}: Gefahren von Fahrzeug ID {p}")

print("\n--- Auswahl Ladegeräte (Charger Selection) ---")
for c in C:
    for i in I:
        if Auswahl_Charger[c, i].X > 0.5:
            print(f"Ladeplatz {i}: Modell {c}")

print("--- SOC für Fahrzeuge wo SOC >0 ---")
for p in P:
    for t in T:
        soc_val = SOC_Zeit_Fahrzeug[p, t].X
        if soc_val > 0:
          print(f"  Zeitintervall {t}; Fahrzeug: {p}; SOC={soc_val:.2f} kWh")

print("\n--- Ladeleistung----")
for p in P:
  for t in T:
    ladeleistung_val = Ladeleistung[p, t].X
    if ladeleistung_val > 0:
      print(f"Fahrzeug ID {p}, Zeitintervall {t}: Ladeleistung={ladeleistung_val:.2f} kW")

print("\n--- Gesamtlast pro Ladeplatz pro Zeitintervall ---")
for t in T:
    # Wir prüfen, ob an irgendeinem Ladeplatz in diesem Intervall geladen wird
    any_load = any(Ladeleistung_Saeule[p, t, i].X > 0.1 for p in P for i in I)

    if any_load:
        print(f"Zeitintervall {t}:")
        for i in I:
            # Berechne die Summe der Last aller LKWs an genau diesem Ladeplatz i
            last_pro_platz = sum(Ladeleistung_Saeule[p, t, i].X for p in P)

            if last_pro_platz > 0.1:
                # Optional: Welcher Charger-Typ steht hier?
                charger_typ = next((c for c in C if Auswahl_Charger[c, i].X > 0.5), "Unbekannt")
                print(f"  - Ladeplatz {i} ({charger_typ}): Gesamtlast = {last_pro_platz:.2f} kW")

print("\n--- Netzausbau (Grid Expansion) ---")
if Ausbau_Netz.X > 0.5:
    print("Netzausbau: Ja")
else:
    print("Netzausbau: Nein")

if (Batterie_kWh.X > 0.5) or (Batterie_kW.X > 0.5):
    print("\n--- Batteriespeicher (Battery Storage) ---")
    print(f"Batteriespeicher gekauft: Ja")
    print(f"  Kapazität: {Batterie_kWh.X:.2f} kWh")
    print(f"  Leistung: {Batterie_kW.X:.2f} kW")
else:
    print("Batteriespeicher gekauft: Nein")


print("\n--- SOC und Laden/Entladen des Speichers (beispielhaft für die ersten 24 Zeitintervalle) ---")
for t in range(0, 24): # Display first 24 time intervals (6 hours)
    soc_storage = SOC_Speicher[t].X
    charge_storage = Laden_Speicher[t].X
    discharge_storage = Entladen_Speicher[t].X
    if soc_storage > 0:
      print(f"  Zeitintervall {t}: SOC_Speicher={soc_storage:.2f} kWh, Laden_Speicher={charge_storage:.2f} kW, Entladen_Speicher={discharge_storage:.2f} kW")

print("\n--- Batterie ---")
print(Batterie_kWh.X)
print(Batterie_kW.X)

In [ ]:
P_peak.X

In [ ]:
print("Model Fertig")

## Excel - Export

In [ ]:
# Export der Optimierungsergebnisse nach Excel
import pandas as pd

excel_path = f"optimierungsergebnisse_{m.MIPGap*100:.2f}%_Gap.xlsx"

# 1. Flottenwahl (welches Modell auf welcher Fahrzeug-ID)
fleet_rows = []
for l in L:
    for p in P:
        fleet_rows.append({
            "truck_model": l,
            "vehicle_id": p,
            "value": float(Flottenwahl[l, p].X)
        })
df_fleet = pd.DataFrame(fleet_rows)

# 2. Routen-Zuordnung (welche Route wird von welcher ID gefahren)
route_assign_rows = []
for r in R:
    for p in P:
        route_assign_rows.append({
            "route_id": r,
            "vehicle_id": p,
            "value": float(Zuordnung_LKW_Route[r, p].X)
        })
df_route_assign = pd.DataFrame(route_assign_rows)

# 3. Einsatz-Variable (Route, LKW-Typ, Fahrzeug-ID)
einsatz_rows = []
for r in R:
    for l in L:
        for p in P:
            einsatz_rows.append({
                "route_id": r,
                "truck_model": l,
                "vehicle_id": p,
                "value": float(Einsatz[r, l, p].X)
            })
df_einsatz = pd.DataFrame(einsatz_rows)

# 4. Auswahl der Charger-Modelle je Säule
charger_rows = []
for c in C:
    for i in I:
        charger_rows.append({
            "charger_model": c,
            "slot": i,
            "value": float(Auswahl_Charger[c, i].X)
        })
df_charger = pd.DataFrame(charger_rows)

# 5. SOC der Fahrzeuge über die Zeit
soc_rows = []
for p in P:
    for t in T:
        soc_rows.append({
            "vehicle_id": p,
            "time_interval": t,
            "soc_kwh": float(SOC_Zeit_Fahrzeug[p, t].X)
        })
df_soc = pd.DataFrame(soc_rows)

# 6. Ladeleistung der Fahrzeuge über die Zeit
power_rows = []
for p in P:
    for t in T:
        power_rows.append({
            "vehicle_id": p,
            "time_interval": t,
            "power_kw": float(Ladeleistung[p, t].X)
        })
df_power = pd.DataFrame(power_rows)

# 7. Zuordnung LKW-Zeitpunkt-Charger
charger_assign_rows = []
for p in P:
    for t in T:
        for i in I:
            charger_assign_rows.append({
                "vehicle_id": p,
                "time_interval": t,
                "slot": i,
                "value": float(Zuordnung_LKW_Zeitpunkt_Charger[p, t, i].X)
            })
df_charger_assign = pd.DataFrame(charger_assign_rows)

# 8. Speicher-Zustände und Leistungen
storage_rows = []
for t in T:
    storage_rows.append({
        "time_interval": t,
        "soc_storage_kwh": float(SOC_Speicher[t].X),
        "charge_kw": float(Laden_Speicher[t].X),
        "discharge_kw": float(Entladen_Speicher[t].X)
    })
df_storage = pd.DataFrame(storage_rows)

# 9. Skalare Größen und Kennzahlen
scalar_rows = [
    {"variable": "Objektwert_gesamt", "value": float(m.objVal)},
    {"variable": "Ausbau_Netz", "value": float(Ausbau_Netz.X)},
    {"variable": "Batterie_kWh", "value": float(Batterie_kWh.X)},
    {"variable": "Batterie_kW", "value": float(Batterie_kW.X)},
    {"variable": "P_peak", "value": float(P_peak.X)}
 ]
df_scalars = pd.DataFrame(scalar_rows)

# -- Kostenaufschlüsselung gemäß Zielfunktion (pro Tag)
# 1) Fahrzeugkosten nach E-Lkw und Verbrenner getrennt
L_e = set(etrucks_specs["truck_model"])
L_d = set(dtrucks_specs["truck_model"])

veh_cost_e = (1/260) * sum(
    float(Flottenwahl[l, p].X) * (capex_veh[l] + opex_veh[l] - thg_e.get(l, 0))
    for l in L_e for p in P if l in L
)

veh_cost_d = (1/260) * sum(
    float(Flottenwahl[l, p].X) * (capex_veh[l] + opex_veh[l] - thg_e.get(l, 0))
    for l in L_d for p in P if l in L
)

veh_cost = veh_cost_e + veh_cost_d

# 2) Ladeinfrastruktur-Kosten
charger_cost = (1/260) * (
    c_capex + sum(
        float(Auswahl_Charger[c, i].X) * (capex_ch[c] + opex_ch[c])
        for c in C for i in I
    )
)

# 3) Speicher-Kosten (Capex + jährliche Abschreibung)
batt_kwh_val = float(Batterie_kWh.X)
batt_kw_val = float(Batterie_kW.X)
battery_cost = (1/260) * (
    (batt_kwh_val * c_capex_bat_kWh) +
    (batt_kw_val * c_capex_bat_kW) +
    (batt_kwh_val * 0.02 * c_capex_bat_kWh) +
    (batt_kw_val * 0.02 * c_capex_bat_kW)
)

# 4) Netzausbau-Kosten
grid_cost = (1/260) * float(Ausbau_Netz.X) * capex_grid_add

# 5) Stromkosten über alle Zeitintervalle mit weiterer Aufschlüsselung
electricity_cost_lkw = 0.0
electricity_cost_store_charge = 0.0
electricity_cost_store_discharge = 0.0
electricity_detail_rows = []
for t in T:
    load_lkw = sum(float(Ladeleistung[p, t].X) for p in P)
    load_store = float(Laden_Speicher[t].X)
    unload_store = float(Entladen_Speicher[t].X)

    cost_lkw = load_lkw * 0.25 * c_energy
    cost_store_charge = load_store * 0.25 * c_energy
    cost_store_discharge = -unload_store * 0.25 * c_energy  # Entladung entlastet das Netz

    electricity_cost_lkw += cost_lkw
    electricity_cost_store_charge += cost_store_charge
    electricity_cost_store_discharge += cost_store_discharge

    electricity_detail_rows.append({
        "time_interval": t,
        "load_lkw_kw": load_lkw,
        "load_store_kw": load_store,
        "unload_store_kw": unload_store,
        "cost_lkw_eur": cost_lkw,
        "cost_store_charge_eur": cost_store_charge,
        "cost_store_discharge_eur": cost_store_discharge,
        "cost_total_eur": cost_lkw + cost_store_charge + cost_store_discharge
    })

electricity_cost_total = electricity_cost_lkw + electricity_cost_store_charge + electricity_cost_store_discharge
df_electricity = pd.DataFrame(electricity_detail_rows)

# 6) Peak-Kosten (auf Basis P_peak; kann leicht von der implementierten Zielfunktion abweichen)
peak_cost = (1/260) * float(P_peak.X) * c_peak

# 7) Mautkosten nur für Diesel-Lkw (ActrosL)
toll_cost = 0.0
if 'ActrosL' in L:
    for r in R:
        for p in P:
            toll_cost += float(Einsatz[r, 'ActrosL', p].X) * dist_toll[r] * c_toll

# 8) Dieselkraftstoffkosten (ActrosL)
diesel_cost = 0.0
if 'ActrosL' in L:
    for r in R:
        for p in P:
            diesel_cost += float(Einsatz[r, 'ActrosL', p].X) * (dist_total[r] * cons_d.get('ActrosL', 0) / 100.0) * diesel

# 9) KFZ-Steuer berechnen (anteilig pro Tag)
cost_tax_val = (1/260) * sum(
    float(Flottenwahl['ActrosL', p].X) * kfz_d.get('ActrosL', 0)
    for p in P if 'ActrosL' in L)

total_cost_decomposed = veh_cost + charger_cost + battery_cost + grid_cost + electricity_cost_total + peak_cost + toll_cost + diesel_cost + cost_tax_val
obj_val = float(m.objVal)


cost_rows = [
    {"component": "Fahrzeuge_E", "cost_per_day": veh_cost_e},
    {"component": "Fahrzeuge_Diesel", "cost_per_day": veh_cost_d},
    {"component": "Fahrzeuge_Gesamt", "cost_per_day": veh_cost},
    {"component": "Ladeinfrastruktur", "cost_per_day": charger_cost},
    {"component": "Speicher", "cost_per_day": battery_cost},
    {"component": "Netzausbau", "cost_per_day": grid_cost},
    {"component": "Strom_LKW", "cost_per_day": electricity_cost_lkw},
    {"component": "Strom_Speicher_Laden", "cost_per_day": electricity_cost_store_charge},
    {"component": "Strom_Speicher_Entladen", "cost_per_day": electricity_cost_store_discharge},
    {"component": "Strom_Gesamt", "cost_per_day": electricity_cost_total},
    {"component": "Peak", "cost_per_day": peak_cost},
    {"component": "Maut_Diesel", "cost_per_day": toll_cost},
    {"component": "Diesel_Kraftstoff", "cost_per_day": diesel_cost},
    {"component": "KFZ_Steuer_Diesel", "cost_per_day": cost_tax_val},
    {"component": "Summe_Komponenten", "cost_per_day": total_cost_decomposed},
    {"component": "Objektwert_Solver", "cost_per_day": obj_val}
]
df_costs = pd.DataFrame(cost_rows)

# Schreiben in eine Excel-Datei mit mehreren Tabellenblättern
with pd.ExcelWriter(excel_path, engine="openpyxl") as writer:
    df_fleet.to_excel(writer, sheet_name="Flottenwahl", index=False)
    df_route_assign.to_excel(writer, sheet_name="Route_LKW", index=False)
    df_einsatz.to_excel(writer, sheet_name="Einsatz", index=False)
    df_charger.to_excel(writer, sheet_name="Charger_Wahl", index=False)
    df_soc.to_excel(writer, sheet_name="SOC_Fahrzeug", index=False)
    df_power.to_excel(writer, sheet_name="Ladeleistung", index=False)
    df_charger_assign.to_excel(writer, sheet_name="LKW_Charger_Zeit", index=False)
    df_storage.to_excel(writer, sheet_name="Speicher", index=False)
    df_scalars.to_excel(writer, sheet_name="Skalare", index=False)
    df_costs.to_excel(writer, sheet_name="Kostenaufschluss", index=False)
    df_electricity.to_excel(writer, sheet_name="Strom_Detail", index=False)

print(f"Optimierungsergebnisse wurden in '{excel_path}' gespeichert.")